In [47]:
import numpy as np
import pandas as pd
import pymysql as db

In [27]:
employees = pd.read_excel('H+ Sport Employees.xlsx',sheet_name='Employees')

In [20]:
employees.head()

,Employee Name,Building,Department,Status,Hire Date,Month,Years,Benefits,Salary,Job Rating,New Salary,Tax Rate,0.0291
0,"Page, Lisa",West,ADC,Full Time,1999-01-11,Jan,25,DMR,24550,1,NaN,NaN,NaN
1,"Taylor, Hector",West,ADC,Half-Time,2011-02-21,Feb,12,DM,26795,4,NaN,NaN,NaN
2,"Dawson, Jonathan",West,ADC,Contract,2007-03-06,Mar,16,NaN,42540,5,NaN,NaN,NaN
3,"Duran, Brian",Taft,ADC,Hourly,2012-08-30,Aug,11,NaN,35680,2,NaN,NaN,NaN
4,"Weber, Larry",Watson,ADC,Full Time,2007-12-31,Dec,16,M,72830,2,NaN,NaN,NaN


In [21]:
employees.shape

(741, 13)

In [28]:
employees.columns

Index(['Employee Name',      'Building',    'Department',        'Status',
           'Hire Date',         'Month',         'Years',      'Benefits',
              'Salary',    'Job Rating',    'New Salary',      'Tax Rate',
                0.0291],
      dtype='object')

In [30]:
remove_columns = ['Job Rating',    'New Salary',      'Tax Rate']
employees.drop(remove_columns, axis=1, inplace=True)

In [31]:
employees.columns

Index(['Employee Name',      'Building',    'Department',        'Status',
           'Hire Date',         'Month',         'Years',      'Benefits',
              'Salary',          0.0291],
      dtype='object')

In [52]:
employees.isna

<bound method DataFrame.isna of         Employee Name Building            Department     Status  Hire Date  \
0          Page, Lisa     West                   ADC  Full Time 1999-01-11   
1      Taylor, Hector     West                   ADC  Half-Time 2011-02-21   
2    Dawson, Jonathan     West                   ADC   Contract 2007-03-06   
3        Duran, Brian     Taft                   ADC     Hourly 2012-08-30   
4        Weber, Larry   Watson                   ADC  Full Time 2007-12-31   
..                ...      ...                   ...        ...        ...   
736       Olsen, Ewan    North  Research/Development   Contract 2007-02-03   
737        Green, Kim   Watson  Research/Development  Half-Time 2000-02-01   
738  Bradshaw, Sheryl    North  Research/Development   Contract 2008-07-10   
739      Riley, David     Taft  Research/Development  Full Time 2010-07-25   
740    Flynn, Melissa   Watson  Research/Development     Hourly 2010-12-31   

    Month  Years Benefits  Sala

In [55]:
# Replace NaN values with a default value 
employees['Benefits'] = employees['Benefits'].fillna('None')

In [61]:
# connect the mysql database at localhost
connection = db.connect(
    host='localhost',
    user='root',
    password='ifeng@8276',
    database='news',
    cursorclass=db.cursors.DictCursor
)

In [50]:
# Create a table named 'employees'
with connection.cursor() as cursor:
    create_table_query ="""
    CREATE TABLE IF NOT EXISTS employees (
        Employee_Name VARCHAR(100) NOT NULL,    
        Building VARCHAR(30),    
        Department VARCHAR(30),        
        Status VARCHAR(30),
        Hire_Date TIMESTAMP NOT NULL,         
        Month VARCHAR(10),         
        Years INT,      
        Benefits VARCHAR(30),
        Salary INT
    );
    """
    cursor.execute(create_table_query)
    connection.commit()

In [54]:
#Insert data into DataFrame
with connection.cursor() as cursor:
    # Correct the number of placeholders to match the columns listed
    insert_query ="""
    INSERT INTO employees (Employee_Name, Building, Department, Status, Hire_Date, Month, Years, Salary)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    for _, row in employees.iterrows():
        # Ensure the tuple has exactly 9 elements, matching the placeholders
        cursor.execute(insert_query, (row['Employee Name'], row['Building'], row['Department'], row['Status'], row['Hire Date'], row['Month'], row['Years'], row['Salary']))
    # Commit the change to the database
    connection.commit()


In [62]:
try:
    with connection.cursor() as cursor:
        # SQL 查询语句
        sql = "SELECT * FROM employees Limit 10"
        cursor.execute(sql)

        # 获取查询结果
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    # 关闭数据库连接
    connection.close()

{'Employee_Name': 'Page, Lisa', 'Building': 'West', 'Department': 'ADC', 'Status': 'Full Time', 'Hire_Date': datetime.datetime(1999, 1, 11, 0, 0), 'Month': 'Jan', 'Years': 25, 'Benefits': 'DMR', 'Salary': 24550}
{'Employee_Name': 'Taylor, Hector', 'Building': 'West', 'Department': 'ADC', 'Status': 'Half-Time', 'Hire_Date': datetime.datetime(2011, 2, 21, 0, 0), 'Month': 'Feb', 'Years': 12, 'Benefits': 'DM', 'Salary': 26795}
{'Employee_Name': 'Page, Lisa', 'Building': 'West', 'Department': 'ADC', 'Status': 'Full Time', 'Hire_Date': datetime.datetime(1999, 1, 11, 0, 0), 'Month': 'Jan', 'Years': 25, 'Benefits': None, 'Salary': 24550}
{'Employee_Name': 'Taylor, Hector', 'Building': 'West', 'Department': 'ADC', 'Status': 'Half-Time', 'Hire_Date': datetime.datetime(2011, 2, 21, 0, 0), 'Month': 'Feb', 'Years': 12, 'Benefits': None, 'Salary': 26795}
{'Employee_Name': 'Dawson, Jonathan', 'Building': 'West', 'Department': 'ADC', 'Status': 'Contract', 'Hire_Date': datetime.datetime(2007, 3, 6, 0, 